In [14]:
! which python

/Users/joe/anaconda3/envs/joe_py36/bin/python


In [34]:
import os
import requests
import json
import pandas as pd
from tabulate import tabulate
def tomtom_category_search_request(api_key, category, top_left=None, btm_right=None):
    # create the directory if it doesn't exist:
    directory = 'output/' + category
    if not os.path.exists(directory):
        os.makedirs(directory)
    url = """
            https://api.tomtom.com/search/2/categorySearch/{category}.json?key={api_key}&topLeft={top_left}&btmRight={btm_right}&countrySet=ZA&limit=100
          """.format(category=category, api_key=api_key, top_left=top_left, btm_right=btm_right)
    filename = 'output/' + category + '/out_' + str(top_left).replace('.','') + '_' + str(btm_right).replace('.','') + '.txt'
    response = requests.get(url)
    data = response.json()
    if 'httpStatusCode' not in data:
        with open(filename, 'w') as outfile:
            json.dump(data, outfile)

        number_pois = len(data['results'])
        poi_data = {}
        # extract the fields we need
        for result in data['results']:
            name = result["poi"]["name"] if "name" in result["poi"] else None
            id = result["id"] if "id" in result else None
            lat = result["position"]["lat"] if "lat" in result["position"] else None
            lon = result["position"]["lon"] if "lon" in result["position"] else None
            categories = [j[0]["name"] for j in [i["names"] for i in result["poi"]["classifications"]]][0]    
            countrySubdivision = result["address"]["countrySubdivision"] if "countrySubdivision" in result["address"] else None
            freeformAddress = result["address"]["freeformAddress"] if "freeformAddress" in result["address"] else None
            poi_data[id] = {
                'name': name,
                'lat': lat,
                'lon': lon,
                'categories': categories,
                'countrySubdivision': countrySubdivision,
                'freeformAddress': freeformAddress
            }

        return {"data": poi_data, "filename": filename, "number_pois": number_pois}
    else:
        return {"error": "error"}

# api_key = '<My API KEY>' # get your API key here: https://developer.tomtom.com/tomtom-maps-apis-developers
api_key = 'I31loGWtvwwXVD9nGFqhIcdv6vfzy1nJ' # get your API key here: https://developer.tomtom.com/tomtom-maps-apis-developers
# result = tomtom_category_search_request(api_key, 'cash dispenser', '-28.416455,21.213253', '-28.457805,21.309467')
# 25.0824302,121.5764799,17z
result = tomtom_category_search_request(api_key, 'high school', '25.0839322,121.5765468,', '25.0339529,121.5651265')
print("Number of POIs found: " + str(result['number_pois']))
print("Output file: " + result['filename'])
# display the first few results
display(pd.DataFrame.from_dict(result['data'], orient='index').head())
df = pd.DataFrame.from_dict(result['data'], orient='index').head()
# print(tabulate(df, headers='keys', tablefmt='psql'))

Number of POIs found: 0
Output file: output/high school/out_250839322,1215765468,_250339529,1215651265.txt


""


In [32]:
def grid_search(top_left, btm_right, api_key, key_word, pois={}):
    print('\nTop-left: ' + top_left)
    print('Bottom-right: ' + btm_right)
    result = tomtom_category_search_request(api_key, key_word, top_left=top_left, btm_right=btm_right)
    num_discovered = 0
    if 'error' not in result:
        ids = set(result['data'].keys())
        new_ids = ids.difference(set(pois.keys()))
        num_discovered = len(new_ids)
        for id in new_ids:
            pois[id] = result['data'][id]
        print("Total POIs found: " + str(len(ids)))
        print("New POIs discovered: " + str(num_discovered))        
        # if the search returned the max number of results, subdivide
        if result["number_pois"] > 98:
            print('Subdividing...')
            # subdivide
            top_leftLat, top_leftLon = top_left.split(',')
            btm_rightLat, btm_rightLon = btm_right.split(',')
            midLat = str((float(top_leftLat) + float(btm_rightLat))/2)
            midLon = str((float(top_leftLon) + float(btm_rightLon))/2)
            new_grid = [
                [[top_leftLat, top_leftLon], [midLat, midLon]], # top left block
                [[top_leftLat, midLon], [midLat, btm_rightLon]], # top right block
                [[midLat, top_leftLon], [btm_rightLat, midLon]], # bottom left block
                [[midLat, midLon], [btm_rightLat, btm_rightLon]] # bottom right block
            ]
            for grid in new_grid:
                top_left = grid[0][0] + ','+ grid[0][1]
                btm_right = grid[1][0] + ',' + grid[1][1]

                grid_search(top_left, btm_right, api_key, key_word, pois)
    return pois
# ct_shopping_centers = grid_search('-33.823464,18.340948', '-34.005645,18.706288', api_key, 'shopping center')
# '25.0824302,121.5764799,', '25.0811691,121.570399'
ct_shopping_centers = grid_search('25.0824302,121.5764799,', '25.0339529,121.5651265', api_key, 'shopping center')
df = pd.DataFrame.from_dict(ct_shopping_centers, orient='index')
display(df)
# print(df.head(10))
# print(tabulate(df.head(10), headers='keys', tablefmt='psql'))


Top-left: 25.0824302,121.5764799,
Bottom-right: 25.0339529,121.5651265
Total POIs found: 0
New POIs discovered: 0


""


In [17]:
df.categories.unique()

array(['shopping center', 'house garden: garden centers services', 'shop',
       'house garden: do-it-yourself centers'], dtype=object)

In [20]:
import numpy as np
np.array(['shop', 'house garden: garden centers services', 'shopping center'],
      dtype=object)

array(['shop', 'house garden: garden centers services', 'shopping center'],
      dtype=object)

In [21]:
df = df.query('categories=="shopping center"')[['name', 'categories', 'lat', 'lon', 'freeformAddress', 'countrySubdivision']].rename(columns={'freeformAddress': 'Address', 'countrySubdivision': 'Province'})

In [22]:
display(df)

,name,categories,lat,lon,Address,Province
ZA/POI/p0/102492,210 on Long Shopping Mall,shopping center,-33.92575,18.41529,"Long Street, Cape Town, City of Cape Town, Wes...",Western Cape
ZA/POI/p0/102580,Milnerton Centre,shopping center,-33.88518,18.49491,"Cape Town, City of Cape Town, Western Cape, 7441",Western Cape
ZA/POI/p0/267496,Convenience Centre,shopping center,-33.88966,18.66717,"Cape Town, City of Cape Town, Western Cape, 7530",Western Cape
ZA/POI/p0/101816,Access Park,shopping center,-33.98793,18.48872,"Cape Town, City of Cape Town, Western Cape, 7708",Western Cape
ZA/POI/p0/102322,Bayside Mall,shopping center,-33.82457,18.48966,"Raats Drive, Cape Town, City of Cape Town, Wes...",Western Cape
...,...,...,...,...,...,...
ZA/POI/p0/101797,AK Centre,shopping center,-33.96804,18.53147,"Klipfontein Road, Cape Town, City of Cape Town...",Western Cape
ZA/POI/p0/102065,Vangate Mall,shopping center,-33.96291,18.53848,"Jakes Gerwel Drive, Cape Town, City of Cape To...",Western Cape
ZA/POI/p0/101581,Access Park Management,shopping center,-33.91841,18.67353,"Van Riebeeck Road 1, Cape Town, City of Cape T...",Western Cape
ZA/POI/p0/102624,Gatesville Shopping Centre,shopping center,-33.97034,18.53522,"Cape Town, City of Cape Town, Western Cape, 7764",Western Cape


In [35]:
import folium
m = folium.Map((25.0339529, 121.5651265), zoom_start=14)
m

In [36]:
import folium
m = folium.Map((25.0339529, 121.5651265), tiles="Cartodb Positron", zoom_start=14)
m